In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/MasterCourse/Multilabel Effects

Mounted at /content/drive
/content/drive/MyDrive/MasterCourse/Multilabel Effects


In [2]:
! pip install -q transformers[sentencepiece] fastbook fastai ohmeow-blurr nbdev datasets==2.14.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from fastai.learner import load_learner

learn = load_learner("models/effect-classifier-stage-1.pkl")

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:455: UserWarning: load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.
If you only need to load model weights and optimizer state, use the safe `Learner.load` instead.
  warn("load_learner` uses Python's insecure pickle module, which can execute malicious arbitrary code when loading. Only load files you trust.\nIf you only need to load model weights and optimizer state, use the safe `Learner.load` instead.")
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
GroupViT models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version.Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.
TAPAS models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability`

In [4]:
# Load effect2idx for label mapping
import json
with open("effect_types_encoded.json", "r") as f:
    effect2idx = json.load(f)
print(f"✅ Loaded {len(effect2idx)} effect labels")

✅ Loaded 19 effect labels


In [5]:
import torch
# Load the saved DataLoaders with weights_only=False, for GPU
dls = torch.load("dataloaders/dls-effect-classifier.pkl", weights_only=False)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [6]:
import torch

# Load the saved DataLoaders with weights_only=False, for CPU
dls = torch.load("dataloaders/dls-effect-classifier.pkl", weights_only=False, map_location=torch.device('cpu'))

In [7]:
# Attach the saved dls back to the learner
learn.dls = dls

In [8]:
# Sample input
sample_text = "I saw impossible geometry and felt a strong sense of ego death while colors melted around me. I felt like I was floating in a sea of colors with vivid patterns swirling around"

# Preprocess the input using the DataLoader
test_dl = learn.dls.test_dl([sample_text], bs=1)

# Run prediction
try:
    # Use get_preds for robust handling of Hugging Face inputs
    preds, _ = learn.get_preds(dl=test_dl)
    # Decode predictions for multi-label classification (threshold=0.2 as in training)
    decoded = [learn.dls.vocab[i] for i, prob in enumerate(preds[0]) if prob > 0.2]

    print("✅ Predicted Effects:")
    print(decoded)

except Exception as e:
    print(f"Error: {e}")
    print("Debugging Info:")
    print(f"DataLoader vocab: {learn.dls.vocab}")
    print(f"Number of categories (c): {learn.dls.c}")
    print("Trying manual preprocessing...")

Error: Exception occured in `TrainEvalCallback` when calling event `before_fit`:
	Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx
Debugging Info:
DataLoader vocab: ['drifting', 'external hallucination', 'geometry', 'memory suppression', 'time distortion', 'unity and interconnectedness', 'anxiety', 'nausea', 'autonomous entities', 'internal hallucination', 'sedation', 'cognitive euphoria', 'physical euphoria', 'stimulation', 'emotion enhancement', 'delusion', 'anxiety suppression', 'spontaneous tactile sensations', 'auditory hallucination']
Number of categories (c): 19
Trying manual preprocessing...


In [ ]:
if torch.cuda.is_available(): # Check if CUDA is even an option (to avoid unnecessary warnings if on CPU-only machine)
    learn.model.cpu() # Move the underlying PyTorch model to CPU
    # If the Learner itself has a 'to_device' method, you might use it
    # learn.to_device('cpu') # This is a common fastai way to move the learner to a device

# Sample input
sample_text = "I saw impossible geometry and felt a strong sense of ego death while colors melted around me. I felt like I was floating in a sea of colors with vivid patterns swirling around"

# Preprocess the input using the DataLoader
test_dl = learn.dls.test_dl([sample_text], bs=1)

# Run prediction
try:
    # Use get_preds for robust handling of Hugging Face inputs
    preds, _ = learn.get_preds(dl=test_dl)
    # Decode predictions for multi-label classification (threshold=0.2 as in training)
    decoded = [learn.dls.vocab[i] for i, prob in enumerate(preds[0]) if prob > 0.2]

    print("✅ Predicted Effects:")
    print(decoded)

except Exception as e:
    print(f"Error: {e}")
    print("Debugging Info:")
    print(f"DataLoader vocab: {learn.dls.vocab}")
    print(f"Number of categories (c): {learn.dls.c}")
    print("Trying manual preprocessing...")